In [63]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader as dl
import pandas as pd
import numpy as np

In [151]:
# Import data #TODO: add data to github and make this a local import
path = '/mnt/c/Users/han/Downloads/merged_obs_sim_features_ref_plus_dist.csv'
raw = pd.read_csv(path)

In [152]:
# Extract relevant columns
x = raw.values[:,[3,4,5,6,7]]
y = raw.values[:,10]
feature_labels = raw.columns[[3,4,5,6,7]]

# Add features
temp_ind = raw.columns.get_loc('vaisala_temp')
temp_col = raw.values[:,temp_ind].reshape(-1,1)
x = np.hstack((x,temp_col**2))
x = np.hstack((x, temp_col**3))
print(x.shape)

(31635, 7)


In [155]:
# Normalize features
m = np.mean(x,0)
m = m.reshape(1,-1)
print(m.shape)
x_mean =  x - m
np.mean(x_mean,0)

(1, 7)


array([3.381478157735609e-13, 2.4791394197879746e-14,
       -1.291584088508599e-13, -1.018269844047855e-11,
       -1.7468588155112523e-12, 1.8502294756278547e-12,
       9.404891082242435e-11], dtype=object)

In [156]:
# Split data
data = x_mean
train_size = int(.8 *len(data))
test_size = len(data) - train_size
val_size = int(.25*train_size)
print("Train: ",train_size," Test: ",test_size," Val: ",val_size)
X_train, X_test = torch.utils.data.random_split(data, [train_size,test_size])
print(len(X_train),len(X_test))
X_train, X_val  = torch.utils.data.random_split(X_train, [train_size-val_size, val_size])
print(len(X_train),len(X_val))

Train:  25308  Test:  6327  Val:  6327
25308 6327
18981 6327


In [157]:
y_train = y[X_train.indices]
y_test  = y[X_test.indices]
y_val   = y[X_val.indices]

In [158]:
# Save data to files:
na = np.array(X_train.dataset.dataset[X_train.indices]).astype(np.float32)
np.save('/home/han/repos/BEACO2N-CO2-Correction/data/X_train', na)

na = np.array(X_test.dataset[X_test.indices]).astype(np.float32)
np.save('/home/han/repos/BEACO2N-CO2-Correction/data/X_test', na)

na = np.array(X_val.dataset.dataset[X_val.indices]).astype(np.float32)
np.save('/home/han/repos/BEACO2N-CO2-Correction/data/X_val', na)

na = np.array(y_train).astype(np.float32)
np.save('/home/han/repos/BEACO2N-CO2-Correction/data/y_train', na)

na = np.array(y_test).astype(np.float32)
np.save('/home/han/repos/BEACO2N-CO2-Correction/data/y_test', na)

na = np.array(y_val).astype(np.float32)
np.save('/home/han/repos/BEACO2N-CO2-Correction/data/y_val', na)

In [159]:
np.mean(X_train,0)

array([0.01789793479552376, 0.02399428198203704, 0.011346996285701223,
       0.00017953742198479255, -0.0008217586024538616, 0.1748679984563908,
       0.4105448534992399], dtype=object)

In [102]:
np.array(y_train).astype(np.float32).shape

(18981,)

In [108]:
dir(X_train.dataset.dataset)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__